MIT License

Copyright (c) 2025 Mohannad Shehadeh

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
include("L-one-two-DTSs.jl");
include("grab-DTS.jl");
include("grab-DTS-slen.jl");
using DelimitedFiles

# Generates header file with minimum scope DTSs:

 Currently supported values of L and M are:
 * if M = 1: all L (specify limit below)
 * if M = 2: all L (specify limit below)
 * if M = 3: all L up to 15
 * if M = 4: all L up to 15
 * if M = 5,6,7,8,9: L = 1

In [2]:
# CREATE C language static array DTS definitions for use with descim-hosc
MAX_L_IN_EXPLICIT_CASE = 25;
L_M_SET = [(L,1) for L in 1:MAX_L_IN_EXPLICIT_CASE];
L_M_SET = [L_M_SET; [(L,2) for L in 1:MAX_L_IN_EXPLICIT_CASE]];
L_M_SET = [L_M_SET; [(L,3) for L in 1:15]];
L_M_SET = [L_M_SET; [(L,4) for L in 1:15]];
L_M_SET = [L_M_SET; [(1,M) for M in [5,6,7,8,9]]];
C_DTS_config = "";
for lmpair in L_M_SET
    local L = lmpair[1];
    local M = lmpair[2];
    local DTS = grabDTS(L,M);
    local SCOPE_MINUS_DTS_c = reverse(maximum(DTS) .- DTS,dims=1);
    C_DTS_config *= 
    (L==1 && M==1 ? "#" : "#el")* 
    "if (L == $L && M == $M)
        //static const int DTS[L][M+1] = " *
        "{"*prod(["{"*prod(string.(DTS[j,:]) .* [i == M+1 ? "" : "," for i in 1:M+1])*"}" for j in 1:L] .* 
    [i == L ? "" : "," for i in 1:L])*"};
        static const int SCOPE_MINUS_DTS_c[L][M+1] = " *
        "{"*prod(["{"*prod(string.(SCOPE_MINUS_DTS_c[j,:]) .* [i == M+1 ? "" : "," for i in 1:M+1])*"}" for j in 1:L] .* 
    [i == L ? "" : "," for i in 1:L])*"};
        #define SCOPE $(maximum(DTS))
"
end
C_DTS_config *= "#endif"
write("dts-defs.h",C_DTS_config)

31008

# Generates header file with minimum scope *and* sum-of-lengths DTSs:

 Currently supported values of L and M are:
 * if M = 1: all L (specify limit below)
 * if M = 2: all L (specify limit below)
 * if M = 3: all L up to 15
 * if M = 4: all L up to 10 excluding 9
 * if M = 5,6,7,8,9: L = 1

In [3]:
# # CREATE C language static array DTS definitions for use with descim-hosc
# MAX_L_IN_EXPLICIT_CASE = 1000;
# L_M_SET = [(L,1) for L in 1:MAX_L_IN_EXPLICIT_CASE];
# L_M_SET = [L_M_SET; [(L,2) for L in 1:MAX_L_IN_EXPLICIT_CASE]];
# L_M_SET = [L_M_SET; [(L,3) for L in 1:15]];
# L_M_SET = [L_M_SET; [(L,4) for L in [1,2,3,4,5,6,7,8,10]]];
# L_M_SET = [L_M_SET; [(1,M) for M in [5,6,7,8,9]]];
# C_DTS_config = "";
# for lmpair in L_M_SET
#     local L = lmpair[1];
#     local M = lmpair[2];
#     local DTS = grabDTSslen(L,M);
#     local SCOPE_MINUS_DTS_c = reverse(maximum(DTS) .- DTS,dims=1);
#     C_DTS_config *= 
#     (L==1 && M==1 ? "#" : "#el")* 
#     "if (L == $L && M == $M)
#         //static const int DTS[L][M+1] = " *
#         "{"*prod(["{"*prod(string.(DTS[j,:]) .* [i == M+1 ? "" : "," for i in 1:M+1])*"}" for j in 1:L] .* 
#     [i == L ? "" : "," for i in 1:L])*"};
#         static const int SCOPE_MINUS_DTS_c[L][M+1] = " *
#         "{"*prod(["{"*prod(string.(SCOPE_MINUS_DTS_c[j,:]) .* [i == M+1 ? "" : "," for i in 1:M+1])*"}" for j in 1:L] .* 
#     [i == L ? "" : "," for i in 1:L])*"};
#         #define SCOPE $(maximum(DTS))
# "
# end
# C_DTS_config *= "#endif"
# write("dts-defs.h",C_DTS_config)